本笔记本展示了如何使用Ollama中的Qwen大模型，进行工具调用。示例中共包含2个工具：

- 天气查询
- 搜索网络

请注意，更强大和更有能力的模型将在复杂的模式和/或多个功能下表现得更好。有关支持的模型和模型变体的完整列表，请参阅[Ollama模型库](https://ollama.ai/library)。

In [4]:
!pip install -q langchain_experimental

In [88]:
import langchain_experimental, langchain_core

for module in (langchain_experimental, langchain_core):
    print(f"{module.__name__:<20}\t{module.__version__}")

langchain_experimental	0.0.59
langchain_core      	0.2.0


# 绑定工具

In [1]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

In [85]:
model = OllamaFunctions(model='qwen:14b', base_url='http://localhost:11434', format='json')

In [81]:
amap_api_key = '替换为自己的高德API Key'
tavily_api_key = '替换为自己的tavily API Key'

def search_web(query, k=5, max_retry=3):
    import os
    from langchain_community.retrievers import TavilySearchAPIRetriever
    
    os.environ["TAVILY_API_KEY"] = tavily_api_key
    retriever = TavilySearchAPIRetriever(k=5)
    documents = retriever.invoke(query)
    # return [{'title': doc.metadata['title'], 'abstract': doc.page_content, 'href': doc.metadata['source'], 'score': doc.metadata['score']} for doc in
    #         documents]
    content = '\n\n'.join([doc.page_content for doc in documents])
    prompt = f"""请将下面这段内容（<<<content>>><<</content>>>包裹的部分）进行总结：
<<<content>>>
{content}
<<</content>>>
"""
    print('prompt:')
    print(prompt)
    
    return model.invoke(prompt).content
    

def get_current_weather(city):
    import requests
    from datetime import datetime
    
    url = f'https://restapi.amap.com/v3/config/district?keywords={city}&key={amap_api_key}'
    resp = requests.get(url)
    # print('行政区划：')
    # print(resp.json())
    
    adcode = resp.json()['districts'][0]['adcode']
    # adcode = '110000'

    url = f'https://restapi.amap.com/v3/weather/weatherInfo?city={adcode}&key={amap_api_key}&extensions=base'
    resp = requests.get(url)
    """
    {'province': '北京',
     'city': '北京市',
     'adcode': '110000',
     'weather': '晴',
     'temperature': '26',
     'winddirection': '西北',
     'windpower': '4',
     'humidity': '20',
     'reporttime': '2024-05-26 13:38:38',
     'temperature_float': '26.0',
     'humidity_float': '20.0'}
    """
    # print('天气：')
    # print(resp.json())
    weather_json = resp.json()['lives'][0]
    return f"{weather_json['city']}{datetime.strptime(weather_json['reporttime'], '%Y-%m-%d %H:%M:%S').strftime('%m月%d日')}{weather_json['weather']}，气温{weather_json['temperature']}摄氏度，{weather_json['winddirection']}风{weather_json['windpower']}级"
    
    
fn_map = {
    'get_current_weather': get_current_weather,
    'search_web': search_web
}

llm_with_tool = model.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "根据城市名获取天气",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "城市名，例如北京"
                    }
                },
                "required": ["city"]
            }
        },
        {
            "name": "search_web",
            "description": "搜索互联网",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要搜素的内容"
                    }
                },
                "required": ["query"]
            }
        },
    ],
    # function_call={"name": "get_current_weather"}
)

# 使用工具

## 天气查询

In [52]:
from langchain_core.messages import HumanMessage
import json

ai_msg = llm_with_tool.invoke("北京今天的天气怎么样")
kwargs = json.loads(ai_msg.additional_kwargs['function_call']['arguments'])

In [53]:
ai_msg.additional_kwargs

{'function_call': {'name': 'get_current_weather',
  'arguments': '{"city": "\\u5317\\u4eac"}'}}

In [55]:
kwargs

{'city': '北京'}

In [58]:
fn_map[ai_msg.additional_kwargs['function_call']['name']](**kwargs)

'北京市05月26日晴，气温25摄氏度，西风≤3级'

## 网络搜索

In [66]:
ai_msg = llm_with_tool.invoke("庆余年2好看吗")
kwargs = json.loads(ai_msg.additional_kwargs['function_call']['arguments'])

In [79]:
ai_msg

AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_web', 'arguments': '{"query": "\\u5e86\\u4f59\\u5e742 \\u597d\\u770b\\u5417 \\u5f71\\u8bc4"}'}}, id='run-15f85676-e800-40e1-88dc-eb99d1994091-0')

In [80]:
kwargs

{'query': '庆余年2 好看吗 影评'}

In [86]:
fn_map[ai_msg.additional_kwargs['function_call']['name']](**kwargs)

prompt:
请将下面这段内容（<<<content>>><<</content>>>包裹的部分）进行总结：
<<<content>>>
庆余年2前5集观后感，优缺点浅评. 1～2集：前两集不太好看，因为看庆2之前我刚把第一季精编版刷完重温了一下剧情，感觉节奏和味道都不太对，没有庆1的节奏感，剧情很平，梗多，但是信息量很低，有些水，假死剧情逻辑上来说圆得很生硬，没什么必要 ...

庆余年2开播这天，我给我自己放了一首《好日子》。 因为点开第一集刚看了个开头，我就知道，属于我的2019年重新杀了回来。 故事的接续感诚意十足，一点开就是张庆被催更，回扣整体的世界观，再跟随故事回到剧中，以同样的节奏同样的视角重现第一季的 ...

微博讨论度也高得夸张，#庆余年 剧王#的词条直接登顶热搜，后面还挂了"爆"字，热度待遇堪比娱乐圈八卦，#庆余年2剧本写了约3年##庆余年 袁泉#等多个相关词条也一起上榜。 微博讨论度 《庆余年2》书接上回，先需要揭开第一季结尾范闲被言冰云刺杀的悬念。

庆余年 第二季电视剧简介和剧情介绍,庆余年 第二季影评、图片、论坛 ... 感觉这几年的期待就是个笑话，真的逃不脱第二季难看得宿命吗？ ... 庆余年2开播这天，我给我自己放了一首《好日子》。 因为点开第一集刚看了个开头，我就知道，属于我的2019年重新杀 ...

观众的打分和评价都一针见血_腾讯新闻. 《庆余年2》第一波真实口碑出炉!. 观众的打分和评价都一针见血. 等了5年，终于等到《 庆余年2 》惊喜开播了。. 而这部剧的首播当晚排面有多大呢？. 全网热度第一，热搜前十八个都和这部剧有关系。. 这个架势，不愧 ...
<<</content>>>



'《庆余年2》的开播似乎备受关注，首播当晚热度爆棚，登顶热搜榜首。观众们的反馈似乎既包含了期待又夹杂着对第一季结束后的剧情走向的揣测。然而，《庆余年2》能否延续并超越前作的口碑，还需时间给出答案。'

# 流式输出

与`langchain_community.llms.ollama.Ollama`中的Ollma类似，此处的OllamaFunctions依然支持流式输出，知识要注意，构造时如果传递了`format`参数，则会流式输出json

In [89]:
model = OllamaFunctions(model='qwen:14b', base_url='http://localhost:11434', format='json')

for chunk in model.stream('介绍一下相对论'):
    print(chunk.content, end='')

{
  "response": [
    {
      "title": "狭义相对论",
      "description": "由爱因斯坦提出，主要阐述物体在高速运动（接近光速）时的物理现象。它包括两个基本原理：相对性原理和光速不变原理。"
    },
    {
      "title": "广义相对论",
      "description": "是狭义相对论的一个自然拓展，由爱因斯坦在1915年提出。它主要描述了引力的本质，将引力视为物体所处的时空弯曲的结果。广义相对论的成功预测包括水星近日点的进动以及光在强引力场中的偏折现象。"
    }
  ]
}

In [90]:
commmon_model = OllamaFunctions(model='qwen:14b', base_url='http://localhost:11434')

for chunk in commmon_model.stream('介绍一下相对论'):
    print(chunk.content, end='')

相对论是物理学中两个重要的理论体系：狭义相对论和广义相对论。

1. **狭义相对论（Special Relativity）**：
   - 由爱因斯坦在1905年提出，主要针对的是非加速运动的观察者。
   - 狭义相对论的主要结论包括：光速不变原理、时间膨胀效应和长度收缩效应。

2. **广义相对论（General Relativity）**：
   - 提出于1915年的爱因斯坦，是对狭义相对论的一个自然推广，用于描述引力。
   - 广义相对论的主要贡献包括：时空弯曲的概念、引力波的预言和黑洞理论的确立。

尽管广义相对论在许多情况下都取得了惊人的成功，但它仍然没有被完全接受。一些物理学家认为可能存在其他形式的引力，这些观点被称为量子引力理论。然而，这些问题目前仍然是物理学界的研究前沿。